In [ ]:
import pandas as pd
import sqlite3
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from data.snowflake_pull import get_snowflake_config, setconnection, run_query_to_df

In [142]:
def find_project_root(start=None, markers=("pyproject.toml", "requirements.txt", ".git", ".env")):
    p = Path(start or os.getcwd()).resolve()
    for parent in (p, *p.parents):
        if any((parent / m).exists() for m in markers):
            return parent
    return Path.cwd().resolve()

ROOT = find_project_root()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

LOCAL_DB_PATH = ROOT / "data" / "inventory_data.db"
LOCAL_DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [143]:
from dotenv import load_dotenv

load_dotenv()


True

In [144]:
print(os.getcwd())

c:\genAIProjects\MBM_WTF\agents


In [ ]:

import data.snowflake_pull as snowflake_pull
import data.sql_lite_store as sql_lite_store

In [146]:
sql_lite_store.list_tables()

['CBM_Max', 'Keppler_Split_Perc', 'demand_data', 'inventory_data']

In [232]:
df_demand = sql_lite_store.load_table("demand_data")
df_Keppler_Split_Perc = sql_lite_store.load_table("Keppler_Split_Perc")
df_CBM_Max = sql_lite_store.load_table("CBM_Max")

df_demand['product_part_number'] = df_demand['product_part_number'].astype(str)
df_Keppler_Split_Perc['ITEM_ID'] = df_Keppler_Split_Perc['ITEM_ID'].astype(str)


In [233]:
df_Keppler_Split_Perc.head()

,ITEM_ID,CARTONIZATION_FLAG,TOTAL_STAT_FCAST,TLA_FCAST,TNY_FCAST,MDT1_FCAST,TLA_FRAC,TNY_FRAC,MDT1_FRAC,upload_timestamp
0,3320782,true,0.001154,0.000568,0.000585,0.000000,0.492474,0.507526,0.000000,2025-11-05 08:32:57.179789
1,215672,true,0.042222,0.020664,0.021558,0.000000,0.489417,0.510583,0.000000,2025-11-05 08:32:57.179789
2,135724,true,33.370429,12.765364,17.491150,3.113915,0.382535,0.524151,0.093314,2025-11-05 08:32:57.179789
3,324508,true,1.252379,0.664741,0.587638,0.000000,0.530782,0.469218,0.000000,2025-11-05 08:32:57.179789
4,1939894,true,0.000999,0.000438,0.000562,0.000000,0.437916,0.562084,0.000000,2025-11-05 08:32:57.179789


In [148]:
df_demand[df_demand['product_part_number'] == '128075']

,product_merch_classification2,product_merch_classification3,product_category_level1,product_category_level2,product_category_level3,parent_product_part_number,product_part_number,product_name,Code,vendor_name,vendor_purchaser_code,Purchase MOQ,Master Case Pack,Case Pk CBM,Final Buy Qty,Vendor Earliest ETD,upload_timestamp
1,Beds & Furniture,Furniture,Cat,"Beds, Crates & Gear",Furniture,155161.0,128075,Frisco 33-in Heavy Duty Sisal Cat Scratching P...,3755,"HANGZHOU TIANYUAN PET PRODUCTS CO., LTD.",JADESCHACK,100,1,0.0409605,7824,2025-11-01 00:00:00,2025-11-04 17:28:02.698947


In [ ]:
config = get_snowflake_config()
conn = setconnection(config)
df_vendor_cbm = run_query_to_df(conn, snowflake_pull.SQL_Vendor_CBM)


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZJdb9owFIb%2FSuRdJ3bCYGABFR%2BritQPVKCM3bnJAbwkdubjEMKvnwll6i5aaXeR87z2Y7%2Bnf3PMM%2B8ABqVWAxIGjHigYp1ItRuQ1fLW7xIPrVCJyLSCAakByc2wjyLPCj4q7V49w%2B8S0HpuI4W8%2BTEgpVFcC5TIlcgBuY35YvRwz6OA8cJoq2OdkXeRzxMCEYx1htdIgtLp7a0tOKVVVQVVK9BmRyPGGGU96qgz8uXKH92dPuBDyr6eeUc4fP7mNpbq8gSfab1eIOR3y%2BXcnz8tlsQbXVUnWmGZg1mAOcgYVs%2F3FwF0BpO77%2BtNUKIPAq0fBqh0tc1ECrHOi9K6PQP3RbeQ0EzvpLv2bDogRSqT1ydT6X06G6e9YtXN1nHnQa4245Z%2BqU%2FtNWjbMYfNNjlNf6Qx8V6uvUbnXmeIJczUuU3rlljU9sPQD9kyYpx1eNQLWlH3J%2FGmrk2phG2SV%2BV4D1Ud6NSKRk0UBf1rTeGY5ie5C7v1ut01rV%2BMHb9RRE3P1ZLLtPDmeDP8vzfo0%2FfZt7F7dE3MpnOdybj2brXJhf24qDAImxWZ%2BNsG5ZALmY2SxACiKyzLdDUxIKybbmtKIHR4OfXf%2BR7%2BAQ%3D%3D&RelayState=ver%

In [150]:
df_demand.columns

Index(['product_merch_classification2', 'product_merch_classification3',
       'product_category_level1', 'product_category_level2',
       'product_category_level3', 'parent_product_part_number',
       'product_part_number', 'product_name', 'Code', 'vendor_name',
       'vendor_purchaser_code', 'Purchase MOQ', 'Master Case Pack',
       'Case Pk CBM', 'Final Buy Qty', 'Vendor Earliest ETD',
       'upload_timestamp'],
      dtype='object')

In [ ]:
df_keppler_filtered = df_Keppler_Split_Perc[['ITEM_ID', 'MDT1_FRAC', 'TLA_FRAC', 'TNY_FRAC']].copy().rename(
    columns={'ITEM_ID':'CHW_SKU_NUMBER'}
)

df_demand1 = df_demand[['product_part_number','Final Buy Qty']].copy().rename(
     columns={'product_part_number':'CHW_SKU_NUMBER', 'Final Buy Qty':'Demand'}) #discarding CHW_PRIMARY_SUPPLIER_NUMBER as its shown to be unreliable

In [248]:
df_vendor_cbm[df_vendor_cbm['CHW_MASTER_CASE_PACK'] ==0]

,MC2_NAME,MC3_NAME,CHW_PRIMARY_SUPPLIER_NAME,PRODUCT,CHW_PRIMARY_SUPPLIER_NUMBER,SKU_NAME,PURCHASE_MOQ,CHW_SKU_NUMBER,CHW_UNITS_PER_CARTON,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM
4,Personalized,Personalized Pet Products,P18662-BOWSERS PET PRODUCTS - V2C,"Frisco Personalized Knife Edge Bed, Dark Grey ...",P18662,"Frisco Personalized Knife Edge Bed, Dark Grey ...",1,311764,0,0,
5,Personalized,Personalized Pet Products,P18662-BOWSERS PET PRODUCTS - V2C,"Frisco Personalized Knife Edge Bed, Indigo Lin...",P18662,"Frisco Personalized Knife Edge Bed, Indigo Lin...",1,311756,0,0,
6,Personalized,Personalized Pet Products,P18662-BOWSERS PET PRODUCTS - V2C,"Frisco Personalized Knife Edge Bed, Indigo Lin...",P18662,"Frisco Personalized Knife Edge Bed, Indigo Lin...",1,311754,0,0,
10,Personalized,Personalized Pet Products,P18532-FUJIFILM NORTH AMERICA CORPORATION - V2C,"Frisco Personalized Rustic Ceramic Dog Bowl, 4...",P18532,"Frisco Personalized Rustic Ceramic Dog Bowl, 4...",1,306764,0,0,
14,Personalized,Personalized Pet Products,P18662-BOWSERS PET PRODUCTS - V2C,"Frisco Personalized Knife Edge Bed, Dark Grey ...",P18662,"Frisco Personalized Knife Edge Bed, Dark Grey ...",1,311762,0,0,
...,...,...,...,...,...,...,...,...,...,...,...
8286,Waste Management,Potty Pads & Diapers,P000486-TIANJIN YIYI HYGIENE PRODUCTS CO. LTD,Disposable Female Diapers,P000486,"Disposable Female Diapers, Large, 12 count",,3612702,0,0,
8287,Waste Management,Potty Pads & Diapers,P000486-TIANJIN YIYI HYGIENE PRODUCTS CO. LTD,Disposable Female Diapers,P000486,"Disposable Female Diapers, Medium, 12 count",,3612694,0,0,
8288,Waste Management,Potty Pads & Diapers,P000486-TIANJIN YIYI HYGIENE PRODUCTS CO. LTD,Disposable Female Diapers,P000486,"Disposable Female Diapers, Small, 12 count",,3612686,0,0,
8289,Waste Management,Potty Pads & Diapers,P000486-TIANJIN YIYI HYGIENE PRODUCTS CO. LTD,Disposable Female Diapers,P000486,"Disposable Female Diapers, Medium, 50 count",,3637670,0,0,


In [219]:
df_demand2 = df_demand1.merge(
    df_keppler_filtered,
    how='left',               # left join keeps all rows from df_demand
    left_on='CHW_SKU_NUMBER',
    right_on='CHW_SKU_NUMBER'
)

df_demand3 = df_demand2.merge(
    df_vendor_cbm,
    how='left',
    on=['CHW_SKU_NUMBER']
)

df_demand3['MDT1_Demand'] = df_demand3['MDT1_FRAC'] * df_demand3['Demand']
df_demand3['TLA_Demand'] = df_demand3['TLA_FRAC'] * df_demand3['Demand']
df_demand3['TNY_Demand'] = df_demand3['TNY_FRAC'] * df_demand3['Demand']

In [241]:
#locating missing records
df_demand_1 = df_demand1.merge(
    df_keppler_filtered,
    how='left',
    on='CHW_SKU_NUMBER',
    indicator=True  # adds a column called "_merge"
)

# Rows where no match was found on the right
skus_w_no_splits = df_demand_1[df_demand_1['_merge'] == 'left_only']


df_demand_2 = df_demand1.merge(
    df_vendor_cbm,
    how='left',
    on=['CHW_SKU_NUMBER'],
    indicator=True  # adds a column called "_merge"
)

# Rows where no match was found on the right
skus_w_no_record_in_plm = df_demand_2[df_demand_2['_merge'] == 'left_only']


In [247]:
df_vendor_cbm[df_vendor_cbm['CHW_SKU_NUMBER'] == '2032758']

,MC2_NAME,MC3_NAME,CHW_PRIMARY_SUPPLIER_NAME,PRODUCT,CHW_PRIMARY_SUPPLIER_NUMBER,SKU_NAME,PURCHASE_MOQ,CHW_SKU_NUMBER,CHW_UNITS_PER_CARTON,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM


In [245]:
skus_w_no_record_in_plm

,CHW_SKU_NUMBER,Demand,MC2_NAME,MC3_NAME,CHW_PRIMARY_SUPPLIER_NAME,PRODUCT,CHW_PRIMARY_SUPPLIER_NUMBER,SKU_NAME,PURCHASE_MOQ,CHW_UNITS_PER_CARTON,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM,_merge
286,876302,1536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
299,2032758,3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [220]:
df_demand3.columns

Index(['CHW_SKU_NUMBER', 'Demand', 'MDT1_FRAC', 'TLA_FRAC', 'TNY_FRAC',
       'MC2_NAME', 'MC3_NAME', 'CHW_PRIMARY_SUPPLIER_NAME', 'PRODUCT',
       'CHW_PRIMARY_SUPPLIER_NUMBER', 'SKU_NAME', 'PURCHASE_MOQ',
       'CHW_UNITS_PER_CARTON', 'CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM',
       'MDT1_Demand', 'TLA_Demand', 'TNY_Demand'],
      dtype='object')

In [222]:
# columns_to_drop = ['Final Buy Qty', 'upload_timestamp', 'MDT1_FRAC', 'TLA_FRAC', 'TNY_FRAC']
# df_demand3 = df_demand3.drop(columns=columns_to_drop)
df_demand4 = df_demand3.drop(columns=['Demand', 'MDT1_FRAC', 'TLA_FRAC', 'TNY_FRAC'], errors='ignore')
# Melt the demand columns from wide to long format
df_demand_long = df_demand4.melt(
    id_vars=[col for col in df_demand4.columns if not col.endswith('_Demand')],
    value_vars=['MDT1_Demand', 'TLA_Demand', 'TNY_Demand'],
    var_name='DEST',
    value_name='Demand'
)

# Clean up the DEST column to remove '_Demand' suffix
df_demand_long['DEST'] = df_demand_long['DEST'].str.replace('_Demand', '')

df_demand_long.head()

df_demand_long.to_csv("df_demand_long.tsv", sep="\t", index=False)



In [169]:
import numpy as np
import pandas as pd
import math

In [170]:


def build_containers(df_long, CBM_MAX, group_cols=('Code','DEST',), soft_overflow=None):
    """
    Assigns container numbers for each group in `group_cols` using FFD.
    Splits rows if a SKU spans multiple containers.
    
    Required columns in df_long:
      - 'Master Case Pack' (integer)
      - 'Case Pk CBM' (CBM per case)
      - 'Demand' (units)
    
    Returns a new dataframe with:
      - 'container' (1,2,3,... within each group)
      - 'cases_assigned'
      - 'cbm_assigned'
    """
    if soft_overflow is None:
        soft_overflow = CBM_MAX  # no soft overflow by default

    req = ['Master Case Pack', 'Case Pk CBM', 'Demand']
    missing = [c for c in req if c not in df_long.columns]
    if missing:
        raise KeyError(f"df_long missing required columns: {missing}")

    # safe numeric conversions
    df = df_long.copy()
    df['Master Case Pack'] = pd.to_numeric(df['Master Case Pack'], errors='coerce').fillna(0).astype(int)
    df['Case Pk CBM']     = pd.to_numeric(df['Case Pk CBM'], errors='coerce')
    df['Demand']          = pd.to_numeric(df['Demand'], errors='coerce').fillna(0.0)

    # compute cases to buy (ceil to MCP multiples)
    df['cases_needed'] = np.where(
        df['Master Case Pack'] > 0,
        np.ceil(df['Demand'] / df['Master Case Pack']).astype(int),
        0
    )
    # rows that can't possibly fit (case CBM > soft cap) are dropped
    df = df[(df['cases_needed'] > 0) & (df['Case Pk CBM'] > 0) & (df['Case Pk CBM'] <= soft_overflow)].copy()

    # sort by decreasing case CBM (FFD)
    sort_cols = list(group_cols) + ['Case Pk CBM']
    df = df.sort_values(sort_cols, ascending=[True]*len(group_cols) + [False])

    out_rows = []

    # process per group (DEST by default)
    for gkey, g in df.groupby(list(group_cols), sort=False):
        # normalize group key to tuple
        gkey_t = gkey if isinstance(gkey, tuple) else (gkey,)

        containers = []  # list of {'cbm_used': float}
        def new_container():
            containers.append({'cbm_used': 0.0})
            return len(containers)  # 1-based container id within group

        for idx, row in g.iterrows():
            cbm_case = float(row['Case Pk CBM'])
            rem_cases = int(row['cases_needed'])

            # try to place all cases for this SKU across containers
            while rem_cases > 0:
                # choose the best container (least free space after adding as many cases as possible)
                best_id = None
                best_free_after = None
                best_fit_cases = 0

                for cid, cont in enumerate(containers, start=1):
                    free_hard = CBM_MAX - cont['cbm_used']
                    fit_hard = int(free_hard // cbm_case)
                    if fit_hard >= 1:
                        # how much space left if we pack as many as possible (bounded by remaining)
                        assign = min(rem_cases, fit_hard)
                        free_after = free_hard - assign * cbm_case
                        if best_id is None or free_after < best_free_after:
                            best_id = cid
                            best_free_after = free_after
                            best_fit_cases = assign

                if best_id is None:
                    # no existing container fits at least one case → open a new container
                    best_id = new_container()
                    cont = containers[best_id - 1]

                    # try hard fit again
                    free_hard = CBM_MAX - cont['cbm_used']
                    fit_hard = int(free_hard // cbm_case)
                    assign = min(rem_cases, max(fit_hard, 0))

                    # if still zero and soft overflow allows a single case, nudge
                    if assign == 0 and cont['cbm_used'] + cbm_case <= soft_overflow:
                        assign = 1
                    if assign == 0:
                        # this SKU case cannot fit even with soft overflow; stop trying this row
                        break
                else:
                    assign = best_fit_cases

                # record assignment
                cbm_assigned = assign * cbm_case
                containers[best_id - 1]['cbm_used'] += cbm_assigned
                rem_cases -= assign

                out = row.to_dict()
                out.update({
                    'cases_assigned': assign,
                    'cbm_assigned': cbm_assigned,
                    'container': best_id,  # within this group
                })
                out_rows.append(out)

    if not out_rows:
        # nothing assigned; return empty frame with expected columns
        return pd.DataFrame(columns=list(df_long.columns) + ['cases_assigned','cbm_assigned','container'])

    result = pd.DataFrame(out_rows)

    # keep original columns + new ones
    keep_cols = list(df_long.columns) + ['cases_assigned', 'cbm_assigned', 'container']
    result = result[keep_cols]

    # nice ordering
    order_front = list(group_cols) + ['container', 'product_part_number'] if 'product_part_number' in result.columns else list(group_cols) + ['container']
    other_cols = [c for c in result.columns if c not in order_front]
    result = result[order_front + other_cols]

    return result



In [74]:
# ---------- Example usage ----------
# Set your container capacity (CBM)
CBM_MAX = 66.0          # example; replace with your real container CBM
SOFT_CAP = CBM_MAX*1.1  # example soft overflow (+10%), or set to CBM_MAX to disable




In [171]:
df[df['Master Case Pack'] == 0]

,product_merch_classification2,product_merch_classification3,product_category_level1,product_category_level2,product_category_level3,parent_product_part_number,product_part_number,product_name,Code,vendor_name,vendor_purchaser_code,Purchase MOQ,Master Case Pack,Case Pk CBM,Vendor Earliest ETD,DEST,Demand,cases_needed


In [178]:
df_demand_long.columns

Index(['CHW_SKU_NUMBER', 'MC2_NAME', 'MC3_NAME', 'CHW_PRIMARY_SUPPLIER_NAME',
       'PRODUCT', 'CHW_PRIMARY_SUPPLIER_NUMBER', 'SKU_NAME', 'PURCHASE_MOQ',
       'CHW_UNITS_PER_CARTON', 'CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM',
       'DEST', 'Demand'],
      dtype='object')

In [180]:
soft_overflow = 66.0
req = ['CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM', 'Demand']

# safe numeric conversions
df = df_demand_long.copy()
mask = df['CHW_MASTER_CARTON_CBM'].notna() & (df['CHW_MASTER_CASE_PACK'] > 0) & (df['Demand'] > 0)
df = df.loc[mask].reset_index(drop=True)
df['CHW_MASTER_CASE_PACK'] = pd.to_numeric(df['CHW_MASTER_CASE_PACK'], errors='coerce').fillna(0).astype(int)
df['CHW_MASTER_CARTON_CBM']     = pd.to_numeric(df['CHW_MASTER_CARTON_CBM'], errors='coerce')
df['Demand']          = pd.to_numeric(df['Demand'], errors='coerce').fillna(0.0)

df['cases_needed'] = np.where(
        df['CHW_MASTER_CASE_PACK'] > 0, np.ceil(df['Demand'] / df['CHW_MASTER_CASE_PACK']).astype(int), 0 )


In [ ]:
excluded = df_demand_long.loc[~mask].copy()
excluded

,CHW_SKU_NUMBER,MC2_NAME,MC3_NAME,CHW_PRIMARY_SUPPLIER_NAME,PRODUCT,CHW_PRIMARY_SUPPLIER_NUMBER,SKU_NAME,PURCHASE_MOQ,CHW_UNITS_PER_CARTON,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM,DEST,Demand
3,353207,Containment & Gear,Gear,P000544-SHANGHAI SENFUL CO LTD,Frisco Bolster Bench Car Seat Cover,P000544,"Frisco Bolster Bench Car Seat Cover, Large, Black",300,0.0,4.0,0.134946,MDT1,0.0
4,210235,Containment & Gear,Containment,B000064-PM&J LLC - DIRECT IMPORT,Frisco Gray Basket Weave Dog Crate Mat,B000064,"Frisco Gray Basket Weave Dog Crate Mat, 48-in",504,0.0,8.0,0.21888,MDT1,0.0
5,169663,Containment & Gear,Gear,P000544-SHANGHAI SENFUL CO LTD,Frisco Water Resistant Hammock Car Seat Cover,P000544,"Frisco Water Resistant Hammock Car Seat Cover,...",300,0.0,12.0,0.06786,MDT1,0.0
14,144994,Containment & Gear,Containment,P000544-SHANGHAI SENFUL CO LTD,Frisco Premium Travel Bag Dog & Cat Carrier,P000544,"Frisco Premium Travel Bag Dog & Cat Carrier, B...",300,10.0,10.0,0.069905,MDT1,0.0
15,258360,Containment & Gear,Gear,P000544-SHANGHAI SENFUL CO LTD,Frisco Quilted Water Resistant Bench Car Seat ...,P000544,Frisco Quilted Water Resistant Bench Car Seat ...,300,0.0,4.0,0.08398,MDT1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
608,876302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TLA,NaN
621,2032758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TLA,NaN
670,256024,Containment & Gear,Containment,B000064-PM&J LLC - DIRECT IMPORT,Frisco Cooling Orthopedic Dog Crate Mat,B000064,"Frisco Cooling Orthopedic Dog Crate Mat, 54-in",256,0.0,8.0,0.767312,TNY,0.0
930,876302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TNY,NaN


In [ ]:
excluded

NameError: name 'excuded' is not defined

In [190]:
df[df['CHW_SKU_NUMBER'] == '862702']

,CHW_SKU_NUMBER,MC2_NAME,MC3_NAME,CHW_PRIMARY_SUPPLIER_NAME,PRODUCT,CHW_PRIMARY_SUPPLIER_NUMBER,SKU_NAME,PURCHASE_MOQ,CHW_UNITS_PER_CARTON,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM,DEST,Demand,cases_needed
150,862702,Waste Management,Potty Pads & Diapers,B3722-CAMBODIA JIANG SU ZHONG HENG PET ARTICLE...,Frisco Indoor Dog Grass Potty Pad with Tray,B3722,"Frisco Indoor Dog Grass Potty Pad with Tray, 2...",102,0.0,6,0.077281,MDT1,14.408724,3
437,862702,Waste Management,Potty Pads & Diapers,B3722-CAMBODIA JIANG SU ZHONG HENG PET ARTICLE...,Frisco Indoor Dog Grass Potty Pad with Tray,B3722,"Frisco Indoor Dog Grass Potty Pad with Tray, 2...",102,0.0,6,0.077281,TLA,65.297046,11
756,862702,Waste Management,Potty Pads & Diapers,B3722-CAMBODIA JIANG SU ZHONG HENG PET ARTICLE...,Frisco Indoor Dog Grass Potty Pad with Tray,B3722,"Frisco Indoor Dog Grass Potty Pad with Tray, 2...",102,0.0,6,0.077281,TNY,22.294230,4


In [184]:
result_rows = []
container_capacity = 66.0

for (code, dest), group in df.groupby(['CHW_PRIMARY_SUPPLIER_NUMBER', 'DEST']):
    # Create a list to track remaining quantities for each SKU
    remaining_quantities = []
    for _, row in group.iterrows():
        remaining_quantities.append({
            'row': row,
            'cases_remaining': row['cases_needed']
        })
        
    current_container = 1
    remaining_capacity = container_capacity
        
    # Keep packing until all SKUs are allocated
    while any(item['cases_remaining'] > 0 for item in remaining_quantities):
        packed_something = False
            
        # Try to fit each SKU into current container
        for item in remaining_quantities:
            if item['cases_remaining'] == 0:
                continue
                
            row = item['row']
            cases_remaining = item['cases_remaining']
            master_case_pack = row['CHW_MASTER_CASE_PACK']
            cbm_per_case = row['CHW_MASTER_CARTON_CBM']
                
            # How many cases can fit in remaining capacity?
            max_cases_by_capacity = int(remaining_capacity / cbm_per_case)
                
            # Round down to nearest Master Case Pack multiple
            cases_in_this_container = (max_cases_by_capacity // master_case_pack) * master_case_pack
                
            # If this SKU can fit at least one Master Case Pack
            if cases_in_this_container > 0:
                cases_to_allocate = min(cases_in_this_container, cases_remaining)
                    
                # Create a row for this allocation
                new_row = row.copy()
                new_row['cases_needed'] = cases_to_allocate
                new_row['total_cbm'] = cases_to_allocate * cbm_per_case
                new_row['container'] = current_container
                result_rows.append(new_row)
                    
                # Update remaining
                item['cases_remaining'] -= cases_to_allocate
                remaining_capacity -= (cases_to_allocate * cbm_per_case)
                packed_something = True
            
        # If nothing could be packed in current container, move to next
        if not packed_something:
            current_container += 1
            remaining_capacity = container_capacity

In [250]:
df_demand_long.columns

Index(['CHW_SKU_NUMBER', 'MC2_NAME', 'MC3_NAME', 'CHW_PRIMARY_SUPPLIER_NAME',
       'PRODUCT', 'CHW_PRIMARY_SUPPLIER_NUMBER', 'SKU_NAME', 'PURCHASE_MOQ',
       'CHW_UNITS_PER_CARTON', 'CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM',
       'DEST', 'Demand'],
      dtype='object')

In [192]:
df_packed.columns

Index(['product_merch_classification2', 'product_merch_classification3',
       'product_category_level1', 'product_category_level2',
       'product_category_level3', 'parent_product_part_number',
       'product_part_number', 'product_name', 'Code', 'vendor_name',
       'vendor_purchaser_code', 'Purchase MOQ', 'Master Case Pack',
       'Case Pk CBM', 'Vendor Earliest ETD', 'DEST', 'Demand', 'cases_needed',
       'total_cbm', 'container'],
      dtype='object')

In [223]:
df_demand_long.columns

Index(['CHW_SKU_NUMBER', 'MC2_NAME', 'MC3_NAME', 'CHW_PRIMARY_SUPPLIER_NAME',
       'PRODUCT', 'CHW_PRIMARY_SUPPLIER_NUMBER', 'SKU_NAME', 'PURCHASE_MOQ',
       'CHW_UNITS_PER_CARTON', 'CHW_MASTER_CASE_PACK', 'CHW_MASTER_CARTON_CBM',
       'DEST', 'Demand'],
      dtype='object')

In [103]:
df_packed= pd.DataFrame(result_rows)

In [186]:
df_packed.to_csv('packed_containers.csv', index=False)

In [189]:
os.getcwd()

'c:\\genAIProjects\\MBM_WTF\\agents'

In [198]:
dup_rows = df_packed[
    df_packed.duplicated(['DEST', 'Code', 'product_part_number'], keep=False)
].sort_values(['DEST','Code','product_part_number'])

repeats = (
    df_packed
      .loc[df_packed['product_part_number'].notna()]
      .groupby(['Code','DEST', 'product_part_number'], dropna=False)
      .size()
      .reset_index(name='rows')
      .query('rows > 1')
      .sort_values(['Code','DEST', 'product_part_number'])
)

In [204]:
missing_rows = df_demand[~df_demand['product_part_number'].isin(df_packed['product_part_number'].dropna().unique())]
missing_skus = missing_rows['product_part_number'].dropna().unique()

In [264]:
sql_lite_store.list_tables()

['CBM_Max', 'Keppler_Split_Perc', 'Vendor_CBM', 'demand_data']

In [211]:
86112+389842

475954

In [224]:
import pandas as pd
# --- Harmonize key names ---
demand = df_demand_long.rename(columns={
    'CHW_SKU_NUMBER': 'product_part_number',
    'CHW_PRIMARY_SUPPLIER_NUMBER': 'Code'
})[['product_part_number','Code','DEST','Demand']]

packed = df_packed[['product_part_number','Code','DEST','cases_needed','Master Case Pack']].copy()

# --- If df_packed can have multiple rows per (SKU, Code, DEST), aggregate first ---
pack_summary = (packed
    .groupby(['product_part_number','Code','DEST'], as_index=False)
    .agg(cases_needed=('cases_needed','sum'),
         master_case_pack=('Master Case Pack','first'))  # MCP should be constant per SKU
)

# Planned units = cases_needed * Master Case Pack
pack_summary['planned_units'] = pack_summary['cases_needed'] * pack_summary['master_case_pack']

# --- Join demand to packed plan ---
cmp = (demand
       .merge(pack_summary, on=['product_part_number','Code','DEST'], how='left'))

# Compute shortfall (positive means under-planned vs demand)
cmp['shortfall_units'] = cmp['Demand'] - cmp['planned_units']

# --- Filter rows where Demand > planned_units ---
# If you want to also catch cases where there is no packed plan (NaN), treat NaN as 0:
cmp['planned_units_filled'] = cmp['planned_units'].fillna(0)
rows_exceed = cmp[cmp['Demand'] > cmp['planned_units_filled']].copy()

# Optional: keep only the most relevant columns
rows_exceed = rows_exceed[[
    'product_part_number','Code','DEST','Demand','planned_units','shortfall_units',
    'cases_needed','master_case_pack'
]].sort_values(['Code','DEST','product_part_number'])


In [231]:
rows_exceed['product_part_number'].unique()

array(['2031742', '2032790', '2085574', '2085606', '112068', '112072',
       '112076', '135716', '135729', '135730', '166447', '191969',
       '191971', '227448', '227451', '227454', '227481', '309791',
       '309795', '309822', '309839', '862702', '112074', '112081',
       '112088', '166334', '166379', '166380', '166388', '166389',
       '166390', '166431', '166437', '166438', '166439', '166446',
       '166457', '166458', '166464', '227447', '227450', '309836',
       '862726', '862734', '984598', '99228', '99229', '99232', '101801',
       '101803', '105074', '128075', '128079', '163745', '163749',
       '211408', '211409', '214354', '214391', '252536', '804630',
       '92257', '92261', '952926', '163747', '146979', '174697', '202534',
       '233614', '264446', '2032726', '2032742', '2032774', '2029310',
       '2031726', '2085510', '2085526', '2085542', '2085558', '2085590',
       '2085622', '166518', '2031758', '2254574', '2257582', '2257870',
       '2257886', '2257902',

In [265]:
# db_path = Path.cwd().parent / "data" / "inventory_data.db"
sql_lite_store.list_tables()
df_keppler =sql_lite_store.load_table("Keppler_Split_Perc")
df_CBM_Max =sql_lite_store.load_table("CBM_Max")


In [269]:
import agents.workingFlow as wf


In [270]:
intermediate_demand = sql_lite_store.load_table("intermediate_demand")
df_CBM_Max = sql_lite_store.load_table("CBM_Max")
df_cbm_capacity_map = wf.build_cbm_capacity_map(intermediate_demand, df_CBM_Max)
output_df_containerPlan = wf.build_containers(intermediate_demand, df_cbm_capacity_map, group_cols=('CHW_PRIMARY_SUPPLIER_NUMBER','DEST'), vendor_col='CHW_PRIMARY_SUPPLIER_NUMBER')
        

In [272]:
output_df_containerPlan.head(3)

,CHW_PRIMARY_SUPPLIER_NUMBER,DEST,container,CHW_SKU_NUMBER,PRODUCT,MC1_NAME,MC2_NAME,MC3_NAME,BRAND,CUSTOMER_EARLIEST_TARGET_DATE,EARLIEST_TARGET_DATE,CHW_MOQ_LEVEL,CHW_OTB,CHW_PRIMARY_SUPPLIER_NAME,CHW_MASTER_CASE_PACK,CHW_MASTER_CARTON_CBM,Demand,cases_needed,cases_assigned,cbm_assigned
0,3722,TLA,1,166310,Frisco Cat Litter Scoop,Hard Goods,Litter Accessories,Waste Disposal,Frisco,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,SKU,0,3722-JIANG SU ZHONG HENG PET ARTICLES JOINT-ST...,48,0.051030,454.046632,10,10,0.510300
1,3722,TNY,1,166310,Frisco Cat Litter Scoop,Hard Goods,Litter Accessories,Waste Disposal,Frisco,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,SKU,0,3722-JIANG SU ZHONG HENG PET ARTICLES JOINT-ST...,48,0.051030,505.953368,11,11,0.561330
2,3755,MDT1,1,252559,Frisco Sofa Pet Bed with Removable Cover,Hard Goods,Beds & Furniture,Beds,Frisco,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00,SKU,0,"3755-HANGZHOU TIANYUAN PET PRODUCTS CO., LTD",1,0.280128,30.993696,31,31,8.683979


In [281]:
sql_lite_store.list_tables()

['CBM_Max',
 'DQ_excluded_skus_w_incomplete_data',
 'DQ_skus_w_no_splits',
 'Keppler_Split_Perc',
 'Vendor_CBM',
 'demand_data',
 'intermediate_demand',
 'output_df_containerPlan',
 'output_df_po_layout']

In [286]:
dd1 = sql_lite_store.load_table("output_df_po_layout")

In [288]:
sum(dd1[dd1['Supplier'] == 'P18120']['Quantity'])

3374